# **Laboratorijska vježba 1**: Zadaci za samostalni rad
## **Podaci iz Svjetskog izvješća o sreći**

Svjetsko izvješće o sreći godišnja je publikacija Mreže rješenja za održivi razvoj Ujedinjenih naroda (engl. *United Nations Sustainable Development Solutions Network*). Sadrži članke i ljestvice nacionalne sreće na temelju ocjena vlastitog života ispitanika, koje izvješće također povezuje s različitim životnim čimbenicima.

U ovoj laboratorijskoj vježbi istražit ćemo sreću u različitim zemljama i povezana obilježja. Skupovi podataka koje ćemo koristiti dostupni su u *Data/happiness2020.csv* i *Data/countries_info.csv*.

U nastavku je dan sažetak stupaca (značajki) u skupu podataka:

**happines2020.csv**

*   country - naziv države
*   happiness_score - ocjena sreće
*   social_support - socijalna podrška (ublažavanje učinaka nejednakosti)
*   healthy_life_expectancy - očekivanji zdravi životni vijek
*   freedom_of_choices - sloboda donošenja životnih izbora
*   generosity - velikodušnost (milosrđe, volonteri)
*   perceptrion_of_corruption - percepcija korupcije
*   world_region - regija države u svijetu

**countries_info.csv**

*   country_name - naziv države
*   area - površina u kvadratnim miljama
*   population - broj stanovnika
*   literacy - postotak pismenih stanovnika




In [ ]:
!head Data/countries_info.csv

In [ ]:
import pandas as pd

DATA_FOLDER = 'Data/'

HAPPINESS_DATASET = DATA_FOLDER+"happiness2020.csv"
COUNTRIES_DATASET = DATA_FOLDER+"countries_info.csv"

## **Zadatak 1: Učitavanje podataka**

Učitajte 2 skupa podataka u Pandas DataFrame-ove (zvane *happiness* i *countries*) te prikažite prve retke. Koristite ispravnu metodu za učitavanje podataka i provjerite jesu li podaci u očekivanom formatu.

In [ ]:
happiness = pd.read_csv(HAPPINESS_DATASET, sep=',')
happiness.head()

In [ ]:
countries = pd.read_csv(COUNTRIES_DATASET, sep=',')
countries.head()

## **Zadatak 2: Spajanje podataka**

Kreirajte DataFrame pod nazivom *country_features* spajanjem prethodno stvorena 2 DataFrame-a. Redak novog DataFrame-a mora opisivati sve značajke koje imamo o nekoj državi.


In [ ]:
new_happiness =happiness.rename(columns={"country":"country_name"})
new_happiness['country_name']= new_happiness['country_name'].str.lower()

country_features = pd.merge(new_happiness, countries, how='outer')
country_features.head()

## **Zadatak 3: Gdje su ljudi sretniji?**

Ispišite prvih 10 država na temelju njihove ocjene sreće (što je veća ocjena, država je sretnija).


In [ ]:
happiness_in_countries = new_happiness.sort_values(ascending=False, by='happiness_score').head(10)
happiness_in_countries

Zanima nas u kojoj su regiji svijeta ljudi sretniji.

Izradite i ispišite DataFrame s:
1.   Prosječnom ocjenom sreće
2.   Brojem zemalja za svaku svjetsku regiju.

Poredajte rezultat da biste prikazali rangiranje sreće.

In [ ]:
happpiness_grouped = new_happiness.groupby(new_happiness.world_region)
happpiness_in_regions = happpiness_grouped['happiness_score'].agg('mean')

happpiness_in_regions

In [ ]:
countries_in_regions = happpiness_grouped.agg('count')
countries_in_regions['country_name']

Najbolje rangirana regija ima samo nekoliko država! Koje su to i koji je njihov rezultat sreće?

In [ ]:
best_region_countries = country_features.groupby("world_region", as_index=False)["happiness_score"].mean().sort_values(by="happiness_score", ascending=False).values[0][0]
country_features.query("world_region==@best_region_countries")   


## **Zadatak 4: Koliko je svijet pismen?**

Ispišite nazive država s razinom pismenosti od 100%.

Za svaku zemlju ispište naziv i regiju svijeta u formatu: {ime regije}-{ime zemlje} ({ocjena sreće}).


In [ ]:
country_literacy_100 = country_features.query('literacy=="100,0"')[['world_region', 'country_name', 'happiness_score']]
country_literacy_100

Koliki je globalni prosjek razine pismenosti?

In [ ]:
global_percent = country_features.literacy.str.replace(',','.').astype(float).mean()

Izračunajte udio zemalja s razinom pismenosti ispod 50%. Ispišite vrijednost u postotcima, formatiranu s 2 decimale.

In [ ]:
#def literacy_50(country_literacy, num_countries):
#  return "%.2f"(country_literacy/num_countries)*100

new_table = country_features.copy()
new_table['literacy'] = new_table['literacy'].str.replace(',','.').astype(float)

literacy_below_50 = new_table.query("literacy < 50.0")
literacy_below_50 = len(literacy_below_50)
num_of_countries = len(new_table)

result="%.2f"%(literacy_below_50 /num_of_countries)*100
result

Ispišite neobrađeni broj i postotak nepismenog svjetskog stanovništva.

In [ ]:
population = new_table['population'].sum()
literate_population = population * (global_percent/100)
illiterate_population=population - literate_population


percent_illiterate = '%.2f'%(illiterate_population/population)
[illiterate_population,percent_illiterate]

## **Zadatak 5: Gustoća naseljenosti**

Dodajte DataFrame-u *country_features* novi stupac zvan *population_density* koji se dobije dijeljenjem stupca *population* sa stupcem *area*.

In [ ]:
country_features["population_density"] = country_features["population"]/country_features["area"]
country_features

Koji je rezultat sreće za 3 države s najnižom gustoćom naseljenosti?

In [ ]:
country_features.sort_values("population_density", ascending=True).head(3).happiness_score

## **Zadatak 6: Zdravi i sretni?**

Iscrtajte na raspršeni dijagram (engl. *scatter plot*) ocjenu sreće (x os) i očekivano trajanje zdravog života (y os).

In [ ]:
country_features.plot.scatter(x='happiness_score', y='healthy_life_expectancy') 